In [1]:
import os

CACHE_DIR = r"C:\Users\vikto\OneDrive\Bureau\Cache"


In [2]:
import requests

seasons = [str(s) for s in range(2022,2026)]


In [3]:
base_url = "https://www.hockey-reference.com"
schedule_links = [f"{base_url}/leagues/NHL_{season}_games.html" for season in seasons]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}


# Takes URL --> file name for caching
def url_to_file(url):
    file = url.split('/')[-1]
    return file

def get_page(url, headers=None):

    file = url_to_file(url)
    filename = os.path.join(CACHE_DIR, file)

    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()

    print(f"File not cached --> Fetching {url}...")
    page = requests.get(url, headers)
    time.sleep(3.1)
    if page.status_code == 200:
        print(f"Successfully fetched page, now caching...")
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(page.text)
            time.sleep(0.4)
            print(f"File Cached. Excellent. Moving right along...")
        return page.text
    else:
        print(f"Couldn't download page. Status code: {page.status_code}")
        print(f"Retry-after: {page.headers.get('Retry-After')}")
        return None







In [4]:
import pandas as pd
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
from io import StringIO
from bs4 import BeautifulSoup
import time

all_games = []
all_box_scores_links = []
for season in schedule_links:
    data = get_page(season, headers)
    soup = BeautifulSoup(data)
    data = StringIO(data)
    games = pd.read_html(data, match='NHL Regular Season Schedule')
    games = games[0]              # Dataframe
    all_games.append(games)
    box_scores_table = soup.find(id='games')  # Get table for box scores links
    box_scores_links = [l.get("href") for l in box_scores_table.find_all("a")]    # Get all links
    box_scores_links = [f"{base_url}{l}" for l in box_scores_links if l and "boxscores/" in l]   # Get absolute URLs
    all_box_scores_links.extend(box_scores_links)
    time.sleep(3)

games = pd.concat(all_games)

In [5]:
games = games.drop(['LOG', 'Unnamed: 6', 'Notes'], axis=1)    # dropping non-relevant columns
games['Home Team Win'] = games['Home'] > games['Visitor']     # creating win column
games['Home Team Win'] = games['Home Team Win'].astype(int)   # Converting all True/False to 1/0
games.rename(columns={'G' : 'Visitor G', 'G.1' : 'Home G'}, inplace=True)


In [6]:
games

,Date,Time,Visitor,Visitor G,Home,Home G,Att.,Home Team Win
0,2021-10-12,7:30 PM,Pittsburgh Penguins,6,Tampa Bay Lightning,2,19092.0,1
1,2021-10-12,10:00 PM,Seattle Kraken,3,Vegas Golden Knights,4,18431.0,1
2,2021-10-13,10:00 PM,Winnipeg Jets,1,Anaheim Ducks,4,16260.0,0
3,2021-10-13,10:00 PM,Chicago Blackhawks,2,Colorado Avalanche,4,18037.0,1
4,2021-10-13,10:00 PM,Vancouver Canucks,2,Edmonton Oilers,3,16034.0,0
...,...,...,...,...,...,...,...,...
1307,2025-04-17,9:30 PM,Calgary Flames,5,Los Angeles Kings,1,18145.0,1
1308,2025-04-17,7:00 PM,Tampa Bay Lightning,0,New York Rangers,4,18006.0,0
1309,2025-04-17,7:00 PM,Carolina Hurricanes,5,Ottawa Senators,7,16193.0,1
1310,2025-04-17,7:00 PM,Washington Capitals,2,Pittsburgh Penguins,5,18348.0,0


In [7]:
#all_box_scores_links
print(f"Length of box_scores_links: {len(all_box_scores_links)}")
print(f"Length of games: {len(games)}")

Length of box_scores_links: 5248
Length of games: 5248


In [8]:
games.head()


,Date,Time,Visitor,Visitor G,Home,Home G,Att.,Home Team Win
0,2021-10-12,7:30 PM,Pittsburgh Penguins,6,Tampa Bay Lightning,2,19092.0,1
1,2021-10-12,10:00 PM,Seattle Kraken,3,Vegas Golden Knights,4,18431.0,1
2,2021-10-13,10:00 PM,Winnipeg Jets,1,Anaheim Ducks,4,16260.0,0
3,2021-10-13,10:00 PM,Chicago Blackhawks,2,Colorado Avalanche,4,18037.0,1
4,2021-10-13,10:00 PM,Vancouver Canucks,2,Edmonton Oilers,3,16034.0,0


In [9]:
# Cell for testing purposes
from team_abbreviations import team_map

all_scores = []
count = 0

for link in all_box_scores_links:
    data = get_page(link)
    soup = BeautifulSoup(data, 'html.parser')

    box = soup.find('div', class_='scorebox')    # Finding scorebox to get names
    teams = box.find_all('strong')
    visitors = teams[0].text.strip()             # extracting visitor team's name
    home = teams[1].text.strip()                 # extracting home team's name

    visitors = team_map.get(visitors)            # Mapping to abbreviation
    home = team_map.get(home)

    data = StringIO(data)

    # Getting scorebox tables
    v_table = pd.read_html(data, attrs={'id' : f'{visitors}_skaters'}, header=1)[0]
    h_table = pd.read_html(data, attrs={'id' : f'{home}_skaters'}, header=1)[0]

    # Find row where the player name/index contains "TOTAL" and get the last one
    v_stats_table = v_table[v_table.iloc[:, 1].str.contains('TOTAL', na=False)].iloc[[-1]]
    h_stats_table = h_table[h_table.iloc[:, 1].str.contains('TOTAL', na=False)].iloc[[-1]]

    # Only keeping relevant columns
    v_stats_table = v_stats_table[['PIM', 'S', 'S%']]
    h_stats_table = h_stats_table[['PIM', 'S', 'S%']]

    # Renaming for processing later
    v_stats_table.rename(columns={'PIM' : 'Visitors PIM', 'S' : 'Visitors S', 'S%' : 'Visitors S%'}, inplace=True)
    h_stats_table.rename(columns={'PIM' : 'Home PIM', 'S' : 'Home S', 'S%' : 'Home S%'}, inplace=True)

    # Resetting indices for concatenating
    v_stats_table.reset_index(drop=True, inplace=True)
    h_stats_table.reset_index(drop=True, inplace=True)

    # Concatenating both tables
    final_table = pd.concat([v_stats_table, h_stats_table], axis=1)

    all_scores.append(final_table)




In [10]:
print(len(all_scores))
scores = pd.concat(all_scores)
scores

5248


,Visitors PIM,Visitors S,Visitors S%,Home PIM,Home S,Home S%
0,2,35,17.1,2,28,7.1
0,8,31,9.7,8,30,13.3
0,14,34,2.9,24,22,18.2
0,12,34,5.9,12,36,11.1
0,11,38,5.3,25,34,5.9
...,...,...,...,...,...,...
0,6,29,17.2,4,31,3.2
0,2,27,0.0,6,22,18.2
0,28,33,15.2,6,32,21.9
0,4,20,10.0,4,38,13.2


In [11]:
combined_games = pd.concat([games.reset_index(drop=True), scores.reset_index(drop=True)], axis=1)
combined_games.rename(columns={'Visitor' : 'Opponent', 'Visitor G' : 'GA', 'Visitors S' : 'SA', 'Visitors S%' : 'SA%'}, inplace=True)
combined_games.rename(columns={'Visitors PIM' : 'Opponent PIM', 'Home G' : 'G', 'Home Team Win' : 'Win/Loss', 'Home PIM' : 'PIM', 'Home S' : 'S', 'Home S%': 'S%'}, inplace=True)

combined_games

,Date,Time,Opponent,GA,Home,G,Att.,Win/Loss,Opponent PIM,SA,SA%,PIM,S,S%
0,2021-10-12,7:30 PM,Pittsburgh Penguins,6,Tampa Bay Lightning,2,19092.0,1,2,35,17.1,2,28,7.1
1,2021-10-12,10:00 PM,Seattle Kraken,3,Vegas Golden Knights,4,18431.0,1,8,31,9.7,8,30,13.3
2,2021-10-13,10:00 PM,Winnipeg Jets,1,Anaheim Ducks,4,16260.0,0,14,34,2.9,24,22,18.2
3,2021-10-13,10:00 PM,Chicago Blackhawks,2,Colorado Avalanche,4,18037.0,1,12,34,5.9,12,36,11.1
4,2021-10-13,10:00 PM,Vancouver Canucks,2,Edmonton Oilers,3,16034.0,0,11,38,5.3,25,34,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,2025-04-17,9:30 PM,Calgary Flames,5,Los Angeles Kings,1,18145.0,1,6,29,17.2,4,31,3.2
5244,2025-04-17,7:00 PM,Tampa Bay Lightning,0,New York Rangers,4,18006.0,0,2,27,0.0,6,22,18.2
5245,2025-04-17,7:00 PM,Carolina Hurricanes,5,Ottawa Senators,7,16193.0,1,28,33,15.2,6,32,21.9
5246,2025-04-17,7:00 PM,Washington Capitals,2,Pittsburgh Penguins,5,18348.0,0,4,20,10.0,4,38,13.2


In [12]:
all_data = []

for link in all_box_scores_links:
    data = get_page(link, headers=headers)
    data = StringIO(data)
    goalie_stats = pd.read_html(data, match='Goalies Table', header=1)   # List of tables (df)
    v_goalie_stats = goalie_stats[0]
    h_goalie_stats = goalie_stats[1]

    # Getting team's primary goalie
    v_goalie_stats = v_goalie_stats[v_goalie_stats['Rk'] == 1]
    h_goalie_stats = h_goalie_stats[h_goalie_stats['Rk'] == 1]

    # Only keeping relevant columns
    v_goalie_stats = v_goalie_stats[['SV%']]
    h_goalie_stats = h_goalie_stats[['SV%']]

    v_goalie_stats.rename(columns={'SV%' : 'Opponent SV%'}, inplace=True)

    goalie_stats = pd.concat([h_goalie_stats, v_goalie_stats], axis=1)
    all_data.append(goalie_stats)




In [13]:
goalie_s = pd.concat(all_data)
all_matches = pd.concat([combined_games.reset_index(drop=True), goalie_s.reset_index(drop=True)], axis=1)
all_matches

,Date,Time,Opponent,GA,Home,G,Att.,Win/Loss,Opponent PIM,SA,SA%,PIM,S,S%,SV%,Opponent SV%
0,2021-10-12,7:30 PM,Pittsburgh Penguins,6,Tampa Bay Lightning,2,19092.0,1,2,35,17.1,2,28,7.1,0.906,0.929
1,2021-10-12,10:00 PM,Seattle Kraken,3,Vegas Golden Knights,4,18431.0,1,8,31,9.7,8,30,13.3,0.903,0.867
2,2021-10-13,10:00 PM,Winnipeg Jets,1,Anaheim Ducks,4,16260.0,0,14,34,2.9,24,22,18.2,0.971,0.818
3,2021-10-13,10:00 PM,Chicago Blackhawks,2,Colorado Avalanche,4,18037.0,1,12,34,5.9,12,36,11.1,0.941,0.889
4,2021-10-13,10:00 PM,Vancouver Canucks,2,Edmonton Oilers,3,16034.0,0,11,38,5.3,25,34,5.9,0.947,0.941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,2025-04-17,9:30 PM,Calgary Flames,5,Los Angeles Kings,1,18145.0,1,6,29,17.2,4,31,3.2,0.828,0.968
5244,2025-04-17,7:00 PM,Tampa Bay Lightning,0,New York Rangers,4,18006.0,0,2,27,0.0,6,22,18.2,1.000,0.818
5245,2025-04-17,7:00 PM,Carolina Hurricanes,5,Ottawa Senators,7,16193.0,1,28,33,15.2,6,32,21.9,0.848,0.781
5246,2025-04-17,7:00 PM,Washington Capitals,2,Pittsburgh Penguins,5,18348.0,0,4,20,10.0,4,38,13.2,0.900,0.868


In [14]:
wanted_order = ['Date', 'Time', 'Home', 'Opponent', 'Att.', 'G', 'GA', 'S', 'S%', 'SV%', 'PIM', 'SA', 'SA%', 'Opponent SV%', 'Opponent PIM', 'Win/Loss']

all_matches = all_matches[wanted_order]
all_matches

,Date,Time,Home,Opponent,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Win/Loss
0,2021-10-12,7:30 PM,Tampa Bay Lightning,Pittsburgh Penguins,19092.0,2,6,28,7.1,0.906,2,35,17.1,0.929,2,1
1,2021-10-12,10:00 PM,Vegas Golden Knights,Seattle Kraken,18431.0,4,3,30,13.3,0.903,8,31,9.7,0.867,8,1
2,2021-10-13,10:00 PM,Anaheim Ducks,Winnipeg Jets,16260.0,4,1,22,18.2,0.971,24,34,2.9,0.818,14,0
3,2021-10-13,10:00 PM,Colorado Avalanche,Chicago Blackhawks,18037.0,4,2,36,11.1,0.941,12,34,5.9,0.889,12,1
4,2021-10-13,10:00 PM,Edmonton Oilers,Vancouver Canucks,16034.0,3,2,34,5.9,0.947,25,38,5.3,0.941,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,2025-04-17,9:30 PM,Los Angeles Kings,Calgary Flames,18145.0,1,5,31,3.2,0.828,4,29,17.2,0.968,6,1
5244,2025-04-17,7:00 PM,New York Rangers,Tampa Bay Lightning,18006.0,4,0,22,18.2,1.000,6,27,0.0,0.818,2,0
5245,2025-04-17,7:00 PM,Ottawa Senators,Carolina Hurricanes,16193.0,7,5,32,21.9,0.848,6,33,15.2,0.781,28,1
5246,2025-04-17,7:00 PM,Pittsburgh Penguins,Washington Capitals,18348.0,5,2,38,13.2,0.900,4,20,10.0,0.868,4,0


In [33]:
# Re-organizing the Dataframe to be from a team's perspective rather than 1 rowgame

home = all_matches.copy()
home.rename(columns={'Home' : 'Team', 'Win/Loss' : 'Result'}, inplace=True)
home['venue'] = 'Home'

away = all_matches.copy()
away['Team'] = away['Opponent']
away['Opponent'] = away['Home']
away['venue'] = 'Away'

# Flip home team/away stats to be from the away team's perspective
away['G'], away['GA'] = away['GA'], away['G']
away['S'], away['SA'] = away['SA'], away['S']
away['S%'], away['SA%'] = away['SA%'], away['S%']
away['SV%'], away['Opponent SV%'] = away['Opponent SV%'], away['SV%']
away['PIM'], away['Opponent PIM'] = away['Opponent PIM'], away['PIM']
away['Win/Loss'] = away['Win/Loss'].apply(lambda x: 1 - x)
away.rename(columns={'Win/Loss' : 'Result'}, inplace=True)
home



,Date,Time,Team,Opponent,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result,venue
0,2021-10-12,7:30 PM,Tampa Bay Lightning,Pittsburgh Penguins,19092.0,2,6,28,7.1,0.906,2,35,17.1,0.929,2,1,Home
1,2021-10-12,10:00 PM,Vegas Golden Knights,Seattle Kraken,18431.0,4,3,30,13.3,0.903,8,31,9.7,0.867,8,1,Home
2,2021-10-13,10:00 PM,Anaheim Ducks,Winnipeg Jets,16260.0,4,1,22,18.2,0.971,24,34,2.9,0.818,14,0,Home
3,2021-10-13,10:00 PM,Colorado Avalanche,Chicago Blackhawks,18037.0,4,2,36,11.1,0.941,12,34,5.9,0.889,12,1,Home
4,2021-10-13,10:00 PM,Edmonton Oilers,Vancouver Canucks,16034.0,3,2,34,5.9,0.947,25,38,5.3,0.941,11,0,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5243,2025-04-17,9:30 PM,Los Angeles Kings,Calgary Flames,18145.0,1,5,31,3.2,0.828,4,29,17.2,0.968,6,1,Home
5244,2025-04-17,7:00 PM,New York Rangers,Tampa Bay Lightning,18006.0,4,0,22,18.2,1.000,6,27,0.0,0.818,2,0,Home
5245,2025-04-17,7:00 PM,Ottawa Senators,Carolina Hurricanes,16193.0,7,5,32,21.9,0.848,6,33,15.2,0.781,28,1,Home
5246,2025-04-17,7:00 PM,Pittsburgh Penguins,Washington Capitals,18348.0,5,2,38,13.2,0.900,4,20,10.0,0.868,4,0,Home


In [43]:
combined = pd.concat([home, away], ignore_index=True)
combined = combined.drop(columns=['Home'])
combined = combined.sort_values(by=['Team']).reset_index(drop=True)
combined

,Date,Time,Team,Opponent,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result,venue
0,2021-12-12,7:00 PM,Anaheim Ducks,St. Louis Blues,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,0,Away
1,2023-01-28,10:30 PM,Anaheim Ducks,Arizona Coyotes,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,0,Home
2,2022-01-14,8:00 PM,Anaheim Ducks,Minnesota Wild,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0,Away
3,2024-11-05,10:00 PM,Anaheim Ducks,Vancouver Canucks,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0,Home
4,2022-10-12,10:00 PM,Anaheim Ducks,Seattle Kraken,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,0,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,8:00 PM,Winnipeg Jets,Tampa Bay Lightning,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1,Home
10492,2025-03-07,7:00 PM,Winnipeg Jets,New Jersey Devils,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1,Away
10493,2023-02-20,7:00 PM,Winnipeg Jets,New York Rangers,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1,Away
10494,2025-03-09,5:00 PM,Winnipeg Jets,Carolina Hurricanes,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,1,Away


In [45]:
# Moving venue column (for readability)

columns = combined.columns.tolist()
columns.remove('venue')
columns.insert(columns.index('Opponent') + 1, 'venue')

combined = combined[columns]
combined


,Date,Time,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,7:00 PM,Anaheim Ducks,St. Louis Blues,Away,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,0
1,2023-01-28,10:30 PM,Anaheim Ducks,Arizona Coyotes,Home,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,0
2,2022-01-14,8:00 PM,Anaheim Ducks,Minnesota Wild,Away,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,10:00 PM,Anaheim Ducks,Vancouver Canucks,Home,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,10:00 PM,Anaheim Ducks,Seattle Kraken,Home,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,8:00 PM,Winnipeg Jets,Tampa Bay Lightning,Home,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,7:00 PM,Winnipeg Jets,New Jersey Devils,Away,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,7:00 PM,Winnipeg Jets,New York Rangers,Away,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,5:00 PM,Winnipeg Jets,Carolina Hurricanes,Away,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,1


In [ ]:
# For computing rolling averages later
# team_df['Avg Goals Last 5'] = team_df['Goals'].rolling(window=5).mean()